In [9]:
# Import libraries
import requests
import time
import pandas as pd
import datetime as dt

In [10]:
subreddit = 'nottheonion'

In [11]:
# Set up dict for info to collect
posts_data = {'created_utc':[],
              'url':[],
              'id':[],
              'num_comments':[],
              'title':[],
              'subreddit':[]
              }

headers = {'User-agent': 'Reddit Post Collector'}

# Set up function to return submission data
def get_submissions(**kwargs):
    res = requests.get("https://api.pushshift.io/reddit/submission/search/",
                       params=kwargs,
                       headers=headers)
    if res.status_code == 200:
        data = res.json()
        return data['data']
    else:
        print(res.status_code)

before = None
count = 0

# Collect up to 10,000 posts as long as there are posts to collect
while True and len(set(posts_data['created_utc'])) < 9500:
    print(count)
    count += 1*500
    
    posts = get_submissions(subreddit=subreddit,
                            size=500,
                            before=before,
                            sort='desc',
                            sort_type='created_utc')
    if not posts:
        break

    for post in posts:
        # Keep track of position for the next call in while loop
        before = post['created_utc']

        # Append info to posts_data dict
        posts_data['created_utc'].append(post['created_utc'])
        posts_data['url'].append(post['full_link'])
        posts_data['id'].append(post['id'])
        posts_data['num_comments'].append(post['num_comments'])
        posts_data['title'].append(post['title'])
        posts_data['subreddit'].append(post['subreddit'])

    time.sleep(1)

# Save posts to dataframe
posts_data = pd.DataFrame(posts_data)

# Create `timestamp` column with `created_utc` translated into readable time
def get_date(created):
    return dt.datetime.fromtimestamp(created)

_timestamp = posts_data['created_utc'].apply(get_date)
posts_data = posts_data.assign(timestamp = _timestamp)

# Export to csv
filetime = time.strftime("%y%m%d_%H%M%S", time.localtime())
posts_data.to_csv('./data/{}_{}.csv'.format(subreddit, filetime), index=False)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [12]:
posts_data.head()

,created_utc,url,id,num_comments,title,subreddit,timestamp
0,1585669214,https://www.reddit.com/r/nottheonion/comments/...,fsf4bh,1,"Uranus blasted a gas bubble 22,000 times bigge...",nottheonion,2020-03-31 20:40:14
1,1585669127,https://www.reddit.com/r/nottheonion/comments/...,fsf3f0,1,"Ancaster man accused of dealing drugs, operati...",nottheonion,2020-03-31 20:38:47
2,1585668798,https://www.reddit.com/r/nottheonion/comments/...,fsezyo,1,"Hamilton cops charge man with selling cocaine,...",nottheonion,2020-03-31 20:33:18
3,1585668732,https://www.reddit.com/r/nottheonion/comments/...,fsez8u,1,"Hamilton cops charge man with selling cocaine,...",nottheonion,2020-03-31 20:32:12
4,1585668552,https://www.reddit.com/r/nottheonion/comments/...,fsex8p,0,r / astrology - The truth about Aquarians (psy...,nottheonion,2020-03-31 20:29:12


In [13]:
posts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
created_utc     10000 non-null int64
url             10000 non-null object
id              10000 non-null object
num_comments    10000 non-null int64
title           10000 non-null object
subreddit       10000 non-null object
timestamp       10000 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 390.7+ KB


In [14]:
posts_data['subreddit'].unique()

array(['nottheonion'], dtype=object)

NameError: name 'de' is not defined